In this file I want to take any N-body coupling and break it down to a series of Jij and hi I can use along with extra ancilla qubits to reconstruct the energy levels of a device

All of this will be in reference the the paper "A Transmon Quantum Annealer: Decomposing Many-Body Ising Constraints Into Pair Interactions"

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import project_lib as mylib
mylib.rl()

In [ ]:
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\014.001.png")
img2=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\014.002.png")
img3=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Jupyter Extra Images\014.003.png")
plt.figure(); plt.imshow(img1);plt.axis('off')
plt.figure(); plt.imshow(img2);plt.axis('off')
plt.figure(); plt.imshow(img3);plt.axis('off')
plt.show('all')


In [ ]:
# this is the original implementation of this
def f1(N,C):
    '''returns the trios of bits connected and the C positivity or negativity for these pairs for these trios'''
    f1= [['c1','c2']+['j1' if N>3 else 'c3']]+[['j%s'%n,'c%s'%(n+2),'j%s'%(n+1)] for n in (np.arange(N-4)+1)]+[['j%s'%(N-3),'c%s'%(N-1),'c%s'%N] if N>3 else 'pass'] 
    if f1[-1]=='pass': f1 = f1[:-1]
    if len(f1)==1: 
        signs = [1.]
    else:
        signs = [1.]*(len(f1)-1)+[-1.]
    #signs = [1] # always starts with 1
    #for i in range(len(f1)-1):
    #    signs += [-np.sign(signs[-1])]
    return f1, signs

def f2(code,N):
    '''takes the codename of the qubit and returns its array position. Only for the junction and computational qubits.'''
    start_position = {'c':0, 'j':N}
    letter = code[0]
    num = int(code[1:])
    return start_position[letter]+num-1
    

def F(N,C):
    '''Takes a number N of the bodies in the interaction and return the corresponding J_ij and h_i 
    required to build these in the Ising model formula as well as the position of each pf the types of bits.'''
    # there are N compuational bits
    M1 = N-3 # there are N-3 joining bits
    M2 = M1+1 # there are M1+1 centre bits
    Q=M1+M2+N # Total qubits
    Trios, signs = f1(N,C)
    print signs
    J_ij = np.zeros((Q,Q))
    h_i = np.zeros(Q)
    for mid_ind,trio in enumerate(Trios): # the mid_ind is the index of the h_i array where the trio
        indexes = [f2(code,N) for code in trio]
        J_ij[(indexes[0],indexes[1])] =+1.*signs[mid_ind] # #################
        J_ij[(indexes[1],indexes[2])] =+1.*signs[mid_ind] # top line of eqn 4
        J_ij[(indexes[2],indexes[0])] =+1.*signs[mid_ind] # #################
        for index in indexes:                           # ###############################
            J_ij[index,N+M1+mid_ind]+=2.*signs[mid_ind] # this loop is the sum in eqn (4)
            h_i[index]+=-1.*signs[mid_ind]              # ###############################
        h_i[N+M1+mid_ind]+=-2.*signs[mid_ind]  # this is the final part of eqn (4)
    J_ij = np.triu(J_ij+np.transpose(J_ij)) # this part is needed since I have been a little uncareful about adding to upper triangle
    return Q, h_i*abs(C), J_ij*abs(C)


### I am still trying to figure out the correct coefficients I need to bring about this multi-body interaction

In [ ]:
# this is the second version of this created after I studied figure 1 in the transmon paper

def f1(N):
    '''returns the trios of bits connected and the C positivity or negativity for these pairs for these trios'''
    f1= [['c1','c2']+['j1' if N>3 else 'c3']]+[['j%s'%n,'c%s'%(n+2),'j%s'%(n+1)] for n in (np.arange(N-4)+1)]+[['j%s'%(N-3),'c%s'%(N-1),'c%s'%N] if N>3 else 'pass'] 
    if f1[-1]=='pass': f1 = f1[:-1]
    return f1

def f2(code,N):
    '''takes the codename of the qubit and returns its array position. Only for the junction and computational qubits.'''
    start_position = {'c':0, 'j':N}
    letter = code[0]
    num = int(code[1:])
    return start_position[letter]+num-1

def f3(index1,index2):
    '''I want this function to return a J_ij for an interaction based on the indices. It will take the indices and 
    use this to figure out the 2 types of qubits in question. Then from this return the J_ij value for that interaction'''
    # there are N compuational bits
    M1 = N-3 # there are N-3 joining bits
    M2 = M1+1 # there are M1+1 centre bits
    Q=M1+M2+N # Total qubits
    ind1 = min(index1,index2)
    ind2 = max(index1,index2)
    
    if ind1<N and ind2<N: # Logical-Logical
        return 1.
    elif ind1<N and N<=ind2<N+M1: # Logical-Adjoining 
        return 1.
    elif ind1<N and N+M1<=ind2<N+M1+M2: # Logical-Centre
        return 2.
    elif N<=ind1<N+M1 and N+M1<=ind2<N+M1+M2: # Adjoining-Centre
        return 2.
    elif N<=ind1<N+M1 and N<=ind2<N+M1: # Adjoining-Adjoining
        return 1.

def F(N,C):
    '''Takes a number N of the bodies in the interaction and return the corresponding J_ij and h_i 
    required to build these in the Ising model formula as well as the position of each pf the types of bits.'''
    # there are N compuational bits
    M1 = N-3 # there are N-3 joining bits
    M2 = M1+1 # there are M1+1 centre bits
    Q=M1+M2+N # Total qubits
    Trios = f1(N)
    J_ij = np.zeros((Q,Q))
    h_i = np.zeros(Q)
    
    # lets sort out all of the local fields
    h_i[:N]=-1      # Logical
    h_i[N:N+M1]=-2  # Adjoining 
    h_i[N+M1:]=-2   # Centre
    
    # lets sort out all of the coupling
    for mid_ind,trio in enumerate(Trios): # the mid_ind is the index of the h_i array where the trio
        indexes = [f2(code,N) for code in trio]
        J_ij[(indexes[0],indexes[1])] += f3(indexes[0],indexes[1])
        J_ij[(indexes[0],indexes[2])] += f3(indexes[0],indexes[2])
        J_ij[(indexes[1],indexes[2])] += f3(indexes[1],indexes[2])
        for index in indexes:
            J_ij[(index,N+M1+mid_ind)] += f3(index,N+M1+mid_ind)
        
    # this part is needed since I have been a little uncareful about adding to upper triangle 
    J_ij = np.triu(J_ij+np.transpose(J_ij))
    return Q, h_i*C, J_ij*C


In [ ]:
N = 5; C = 1.
qubits, h_i, J_ij=F(N,C)   
print h_i, '\n\n', J_ij
'''h_i = np.asarray([-1., -1., -1., -1., -1., -2., -2., -2., -2., -2.]) 
J_ij = np.asarray(
[[ 0.,  1.,  0.,  0.,  0.,  1.,  0.,  2.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  2.,  0.,  0.],
 
 [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  2.,  0.],
 
 [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  2.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  2.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  2.,  2.,  2.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  2.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]
)'''
test = mylib.Anneal(qubits,[h_i,J_ij])
eigvals,P,P_in = mylib.Diagonaliser(test.Hp)

In [ ]:
from tabulate import tabulate

def bit_tab(Hp,qubits,N):
    '''given a state this fucntion prints out the table of the probability of measuring this state in it's eigenfunctions'''
    headers = ['Bit Solution','Eigenvalue','Parity']
    all_bits = []
    all_arrows = []
    table = []
    parity = []
    eigvals = []
    import sys
    ud = [u"\u2191", u"\u2193"]
    for i in range(2**qubits):
        # local calculations
        bits =  [(i/2**j)%2 for j in range(qubits)[::-1]]
        arrows = [ud[i] for i in bits]
        v = mylib.make_eigen_vector(bits)
        eigval =  (v*Hp*v.getH()).item(0)
         #list appending
        parity.append(np.prod(np.asarray([1-2*b for b in bits])[:N]))
        all_bits.append(bits)
        all_arrows.append(arrows)
        eigvals.append(eigval)
        #add to table
        table.append([' - '.join(arrows[:N])+' : '+' '.join(arrows[N:2*N-3]) +' : '+' '.join(arrows[2*N-3:3*N-5]),eigval,parity[-1]])
    # sort the table 
    order = np.asarray([entry[1] for entry in table]).argsort()
    table = [table[i] for i in order]
    print tabulate(table, headers, tablefmt="grid")

bit_tab(test.Hp,qubits,N)

In [ ]:
# this cell appears to work the way I have derived in my notes. So the mistake must be in the above cell
bit = [0,1,0,0]
v = mylib.make_eigen_vector(bit)
eigval =  (v*test.Hp*v.getH()).item(0)
bit_tab([eigval],v,N)

In [ ]:
def f1(N):
    '''returns the trios of bits connected'''
    f1= [['c1','c2']+['j1' if N>3 else 'c3']]+[['j%s'%n,'c%s'%(n+2),'j%s'%(n+1)] for n in (np.arange(N-4)+1)]+[['j%s'%(N-3),'c%s'%(N-1),'c%s'%N] if N>3 else 'pass'] 
    if f1[-1]=='pass': f1 = f1[:-1]
    return f1
f1(5)